In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt
import time

# Using mediapipe holistic, full body pose estimation

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_custom_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(255, 255, 204), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(225, 225, 225), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(204, 153, 255), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    

# Get landmarks keypoints from frames

In [6]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
  
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

# Load data

In [7]:
DATA_PATH = os.path.join('points_Data')

actions = np.array(['hello', 'thank you', 'i am', 'w', 'a'])
#['##', hello', 'thank you', 'i love you', 'i am', 'w', 'a', 'i', 'l']
no_sequences = 50

sequence_length = 30

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
actions

array(['hello', 'thank you', 'i am', 'w', 'a'], dtype='<U9')

In [11]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in  range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [14]:
len(sequences)

250

# Tensor Flow Model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 3s 63ms/step - loss: 1.8671 - categorical_accuracy: 0.2194
Epoch 2/2000
8/8 [==============================] - 1s 70ms/step - loss: 1.6743 - categorical_accuracy: 0.1519
Epoch 3/2000
8/8 [==============================] - 1s 70ms/step - loss: 1.5276 - categorical_accuracy: 0.2532
Epoch 4/2000
8/8 [==============================] - 1s 73ms/step - loss: 1.6368 - categorical_accuracy: 0.1983
Epoch 5/2000
8/8 [==============================] - 1s 70ms/step - loss: 1.5782 - categorical_accuracy: 0.3207
Epoch 6/2000
8/8 [==============================] - 1s 71ms/step - loss: 1.5111 - categorical_accuracy: 0.4008
Epoch 7/2000
8/8 [==============================] - 1s 73ms/step - loss: 1.3695 - categorical_accuracy: 0.4008
Epoch 8/2000
8/8 [==============================] - 1s 72ms/step - loss: 1.5393 - categorical_accuracy: 0.1941
Epoch 9/2000
8/8 [==============================] - 1s 75ms/step - loss: 1.5838 - categorical_accuracy: 0.2110
E

8/8 [==============================] - 1s 80ms/step - loss: 0.4419 - categorical_accuracy: 0.7932
Epoch 75/2000
8/8 [==============================] - 1s 83ms/step - loss: 0.5225 - categorical_accuracy: 0.7342
Epoch 76/2000
8/8 [==============================] - 1s 82ms/step - loss: 0.3634 - categorical_accuracy: 0.8059
Epoch 77/2000
8/8 [==============================] - 1s 81ms/step - loss: 0.2860 - categorical_accuracy: 0.8903
Epoch 78/2000
8/8 [==============================] - 1s 81ms/step - loss: 0.2250 - categorical_accuracy: 0.8186
Epoch 79/2000
8/8 [==============================] - 1s 83ms/step - loss: 0.1671 - categorical_accuracy: 0.9620
Epoch 80/2000
8/8 [==============================] - 1s 82ms/step - loss: 0.3168 - categorical_accuracy: 0.8481
Epoch 81/2000
8/8 [==============================] - 1s 83ms/step - loss: 0.1933 - categorical_accuracy: 0.9325
Epoch 82/2000
8/8 [==============================] - 1s 81ms/step - loss: 0.1005 - categorical_accuracy: 0.9747
Epoch 

8/8 [==============================] - 1s 92ms/step - loss: 0.6689 - categorical_accuracy: 0.8143
Epoch 147/2000
8/8 [==============================] - 1s 89ms/step - loss: 0.4650 - categorical_accuracy: 0.7764
Epoch 148/2000
8/8 [==============================] - 1s 91ms/step - loss: 0.3328 - categorical_accuracy: 0.7975
Epoch 149/2000
8/8 [==============================] - 1s 93ms/step - loss: 0.2412 - categorical_accuracy: 0.9114
Epoch 150/2000
8/8 [==============================] - 1s 92ms/step - loss: 0.1266 - categorical_accuracy: 0.9620
Epoch 151/2000
8/8 [==============================] - 1s 92ms/step - loss: 0.0612 - categorical_accuracy: 0.9873
Epoch 152/2000
8/8 [==============================] - 1s 94ms/step - loss: 0.0896 - categorical_accuracy: 0.9747
Epoch 153/2000
8/8 [==============================] - 1s 98ms/step - loss: 0.0898 - categorical_accuracy: 0.9789
Epoch 154/2000
8/8 [==============================] - 1s 95ms/step - loss: 0.0500 - categorical_accuracy: 0.978

8/8 [==============================] - 1s 100ms/step - loss: 0.1255 - categorical_accuracy: 0.9916
Epoch 219/2000
8/8 [==============================] - 1s 99ms/step - loss: 0.0629 - categorical_accuracy: 0.9958
Epoch 220/2000
8/8 [==============================] - 1s 97ms/step - loss: 0.0477 - categorical_accuracy: 0.9958
Epoch 221/2000
8/8 [==============================] - 1s 99ms/step - loss: 0.0365 - categorical_accuracy: 0.9958
Epoch 222/2000
8/8 [==============================] - 1s 104ms/step - loss: 0.0594 - categorical_accuracy: 0.9789
Epoch 223/2000
8/8 [==============================] - 1s 105ms/step - loss: 0.0193 - categorical_accuracy: 0.9958
Epoch 224/2000
8/8 [==============================] - 1s 112ms/step - loss: 0.0207 - categorical_accuracy: 1.0000
Epoch 225/2000
8/8 [==============================] - 1s 121ms/step - loss: 0.0247 - categorical_accuracy: 0.9958
Epoch 226/2000
8/8 [==============================] - 1s 116ms/step - loss: 0.3962 - categorical_accuracy:

8/8 [==============================] - 1s 112ms/step - loss: 1.2906e-04 - categorical_accuracy: 1.0000
Epoch 289/2000
8/8 [==============================] - 1s 111ms/step - loss: 1.2502e-04 - categorical_accuracy: 1.0000
Epoch 290/2000
8/8 [==============================] - 1s 107ms/step - loss: 1.2377e-04 - categorical_accuracy: 1.0000
Epoch 291/2000
8/8 [==============================] - 1s 110ms/step - loss: 1.2003e-04 - categorical_accuracy: 1.0000
Epoch 292/2000
8/8 [==============================] - 1s 112ms/step - loss: 1.1645e-04 - categorical_accuracy: 1.0000
Epoch 293/2000
8/8 [==============================] - 1s 107ms/step - loss: 1.1176e-04 - categorical_accuracy: 1.0000
Epoch 294/2000
8/8 [==============================] - 1s 108ms/step - loss: 1.0869e-04 - categorical_accuracy: 1.0000
Epoch 295/2000
8/8 [==============================] - 1s 111ms/step - loss: 1.0577e-04 - categorical_accuracy: 1.0000
Epoch 296/2000
8/8 [==============================] - 1s 108ms/step - l

...

Epoch 1668/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.6659e-08 - categorical_accuracy: 1.0000
Epoch 1669/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.6156e-08 - categorical_accuracy: 1.0000
Epoch 1670/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.5653e-08 - categorical_accuracy: 1.0000
Epoch 1671/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.6156e-08 - categorical_accuracy: 1.0000
Epoch 1672/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.5653e-08 - categorical_accuracy: 1.0000
Epoch 1673/2000
8/8 [==============================] - 1s 119ms/step - loss: 2.5653e-08 - categorical_accuracy: 1.0000
Epoch 1674/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.5150e-08 - categorical_accuracy: 1.0000
Epoch 1675/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.5653e-08 - categorical_accuracy: 1.0000
Epoch 1676/2000
8/8 [===========================

8/8 [==============================] - 1s 121ms/step - loss: 2.1629e-08 - categorical_accuracy: 1.0000
Epoch 1737/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.2132e-08 - categorical_accuracy: 1.0000
Epoch 1738/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.1629e-08 - categorical_accuracy: 1.0000
Epoch 1739/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.1629e-08 - categorical_accuracy: 1.0000
Epoch 1740/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.1126e-08 - categorical_accuracy: 1.0000
Epoch 1741/2000
8/8 [==============================] - 1s 121ms/step - loss: 2.1126e-08 - categorical_accuracy: 1.0000
Epoch 1742/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.1126e-08 - categorical_accuracy: 1.0000
Epoch 1743/2000
8/8 [==============================] - 1s 120ms/step - loss: 2.1126e-08 - categorical_accuracy: 1.0000
Epoch 1744/2000
8/8 [==============================] - 1s 122ms/

Epoch 1805/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.7605e-08 - categorical_accuracy: 1.0000
Epoch 1806/2000
8/8 [==============================] - 1s 120ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1807/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1808/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1809/2000
8/8 [==============================] - 1s 121ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1810/2000
8/8 [==============================] - 1s 120ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1811/2000
8/8 [==============================] - 1s 120ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1812/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.8108e-08 - categorical_accuracy: 1.0000
Epoch 1813/2000
8/8 [===========================

8/8 [==============================] - 1s 123ms/step - loss: 1.6599e-08 - categorical_accuracy: 1.0000
Epoch 1874/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.5593e-08 - categorical_accuracy: 1.0000
Epoch 1875/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.5593e-08 - categorical_accuracy: 1.0000
Epoch 1876/2000
8/8 [==============================] - 1s 120ms/step - loss: 1.6096e-08 - categorical_accuracy: 1.0000
Epoch 1877/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.8611e-08 - categorical_accuracy: 1.0000
Epoch 1878/2000
8/8 [==============================] - 1s 124ms/step - loss: 1.6599e-08 - categorical_accuracy: 1.0000
Epoch 1879/2000
8/8 [==============================] - 1s 125ms/step - loss: 1.5090e-08 - categorical_accuracy: 1.0000
Epoch 1880/2000
8/8 [==============================] - 1s 122ms/step - loss: 1.6599e-08 - categorical_accuracy: 1.0000
Epoch 1881/2000
8/8 [==============================] - 1s 124ms/

Epoch 1942/2000
8/8 [==============================] - 1s 125ms/step - loss: 8.0479e-09 - categorical_accuracy: 1.0000
Epoch 1943/2000
8/8 [==============================] - 1s 124ms/step - loss: 8.5509e-09 - categorical_accuracy: 1.0000
Epoch 1944/2000
8/8 [==============================] - 1s 125ms/step - loss: 9.0539e-09 - categorical_accuracy: 1.0000
Epoch 1945/2000
8/8 [==============================] - 1s 125ms/step - loss: 9.0539e-09 - categorical_accuracy: 1.0000
Epoch 1946/2000
8/8 [==============================] - 1s 126ms/step - loss: 8.5509e-09 - categorical_accuracy: 1.0000
Epoch 1947/2000
8/8 [==============================] - 1s 122ms/step - loss: 8.0479e-09 - categorical_accuracy: 1.0000
Epoch 1948/2000
8/8 [==============================] - 1s 124ms/step - loss: 8.5509e-09 - categorical_accuracy: 1.0000
Epoch 1949/2000
8/8 [==============================] - 1s 124ms/step - loss: 8.5509e-09 - categorical_accuracy: 1.0000
Epoch 1950/2000
8/8 [===========================

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [21]:
res = model.predict(X_test)
len(res)

1/1 [==============================] - 1s 677ms/step


13

In [22]:
actions[np.argmax(res[0])]

'w'

In [23]:
actions[np.argmax(y_test[0])]

'w'

In [65]:
model.save('sign_Language.keras')

In [25]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [26]:
yhat = model.predict(X_train)

8/8 [==============================] - 1s 64ms/step


In [27]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [28]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[190,   0],
        [  0,  47]],

       [[189,   0],
        [  0,  48]],

       [[189,   0],
        [  0,  48]],

       [[190,   0],
        [  0,  47]],

       [[190,   0],
        [  0,  47]]], dtype=int64)

In [29]:
accuracy_score(ytrue, yhat)

1.0

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,117,22),(21,23,245)]
#, (16,117,245),(21,23,245),(16,22,33),(16,117,22),(16,0,245),(23,117,245)
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [68]:
#plt.figure(figsize=(18,18))
#plt.imshow(prob_viz(res, actions, image, colors))

# Test the model

In [69]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import load_model
model=load_model('sign_Language.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [ ]:
sequence = []
sentence = []
threshold = 0.7


cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    prev_keypoints = None  # Store the previous keypoints
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
         
        draw_custom_landmarks(image, results)
        
        # Predict only when there's a significant change in keypoints
        keypoints = get_keypoints(results)
        
        if prev_keypoints is not None:
            # Calculate the Euclidean distance between the current and previous keypoints
            euclidean_distance = np.linalg.norm(np.array(prev_keypoints) - np.array(keypoints))
            #print(euclidean_distance)
            # You can set a threshold for the distance below which you don't make predictions
            if euclidean_distance > 0.2:
                sequence.append(keypoints)
                sequence = sequence[-30:]
        
        prev_keypoints = keypoints
        
        
        #predict
        
#         keypoints = get_keypoints(results)
#         sequence.append(keypoints)
#         sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = []
        if res.any():
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                        
                else:
                    sentence.append(actions[np.argmax(res)])
                  

    
        if len(sentence) > 5:
            sentence = sentence[-5:]
        # Viz probabilities
        
        image = prob_viz(res, actions, image, colors)
            
        
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Camera", image)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
    

1/1 [==============================] - 0s 18ms/step


In [42]:
cap.release()
cv2.destroyAllWindows()